# Thrive Data Modelling assignment notebook

In [302]:
import pandas as pd
import sqlite3

In [303]:
connection = sqlite3.connect("thrive_test_db.db")

In [304]:
# Create a cursor object
cursor = connection.cursor()

In [305]:
# Get all table objects
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

# Fetch all the tables
tables = cursor.fetchall()
print("Tables in the database:")
for table in tables:
    print(table[0])

Tables in the database:
conversation_start
conversation_parts
users
consolidated_messages


In [310]:
users_table = "users"
conversations_start = "conversation_start"
conversations_part = "conversation_parts"

query = f"SELECT * FROM {users_table};"
users_dict = pd.read_sql_query(query, connection)

query = f"SELECT * FROM {conversations_start};"
conversations_start = pd.read_sql_query(query, connection)

query = f"SELECT * FROM {conversations_part};"
conversations_part = pd.read_sql_query(query, connection)

In [361]:
conversations_part

,created_at,conversation_id,part_type,id,conv_dataset_email,message
0,1679498651,10065,comment,20482598151,robert.flores@example.com,Memory support seek ok size. Politics generati...
1,1679498716,10065,assignment,20482622915,anthony.callahan@example.com,Both necessary test available. Until continue ...
2,1679498908,10065,comment,20482707054,alexandra.richardson@example.com,Turn important similar arrive. Marriage agree ...
3,1679499097,10065,comment,20482788524,anthony.callahan@example.com,Measure anything production cup explain well. ...
4,1679499376,10065,comment,20482911573,alexandra.richardson@example.com,Rich already rise head then executive. They ha...
...,...,...,...,...,...,...
80,1726494982,102837,open,21904986991,teresa.mitchell@example.com,Have candidate campaign area skill news no beh...
81,1726495021,102837,comment,21904986992,anthony.callahan@example.com,Play oil focus whole place yeah. Worry list bi...
82,1726495022,102837,close,21904986994,anthony.callahan@example.com,Bag certain between air week subject answer pa...
83,1726495252,102837,open,21904986998,teresa.mitchell@example.com,When boy reason enjoy region choose career. In...


In [369]:


query = f"""    WITH customer_map AS (
  SELECT
    conv.conversation_id,
    MAX(u.id) AS customer_user_id  -- picking one customer per thread deterministically
  FROM (
    SELECT cp.conversation_id, cp.conv_dataset_email AS email FROM conversation_parts cp
    UNION
    SELECT cs.id AS conversation_id, cs.conv_dataset_email AS email FROM conversation_start cs
  ) AS conv
  JOIN users u ON conv.email = u.email
  WHERE u.is_customer = 1
  GROUP BY conv.conversation_id
)
SELECT * FROM customer_map;
  
  """

mapping = pd.read_sql_query(query, connection)

mapping

,conversation_id,customer_user_id
0,10065,4
1,10106,7
2,10222,5
3,10308,12
4,10342,11
5,10350,2
6,10404,0
7,100120,1
8,102013,3
9,102837,10


In [362]:
query = """

WITH customer_map AS (
  SELECT
    conv.conversation_id,
    MAX(u.id) AS customer_user_id  -- picking one customer per thread deterministically
  FROM (
    SELECT cp.conversation_id, cp.conv_dataset_email AS email FROM conversation_parts cp
    UNION
    SELECT cs.id AS conversation_id, cs.conv_dataset_email AS email FROM conversation_start cs
  ) AS conv
  JOIN users u ON conv.email = u.email
  WHERE u.is_customer = 1
  GROUP BY conv.conversation_id
),
 
final_tab as (
SELECT 
    cm.customer_user_id as user_id,
    cs.id AS conversation_id,
    cs.conv_dataset_email AS email,
    cs.message AS message,
    'open' AS message_type,
     0 AS part_id,
    cs.created_at
FROM conversation_start cs
LEFT JOIN customer_map cm ON cs.id = cm.conversation_id

UNION ALL

SELECT 
    
    cm.customer_user_id as user_id,
    cp.conversation_id,
    cp.conv_dataset_email AS email,
    cp.message,
    cp.part_type AS message_type,
    cp.id AS part_id,
    cp.created_at
FROM conversation_parts cp
LEFT JOIN customer_map cm ON cp.conversation_id = cm.conversation_id
)

select row_number() over (order by conversation_id, created_at) as id, *  from final_tab
 
"""
consolidated_messages_df = pd.read_sql_query(query, connection)

In [367]:

consolidated_messages_df.to_csv('consolidated_messages_final.csv')

In [370]:
consolidated_messages_df.to_sql("consolidated_messages", connection, index=False, if_exists="replace")


95

In [371]:
# Get all table objects
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

# Fetch all the tables
tables = cursor.fetchall()
print("Tables in the database:")
for table in tables:
    print(table[0])

Tables in the database:
conversation_start
conversation_parts
users
consolidated_messages


# New table created in the db

In [299]:
consolidated_messages_df.isnull().sum()

id                 0
user_id            0
conversation_id    0
email              0
message            0
message_type       0
created_at         0
dtype: int64

In [365]:
consolidated_messages_df.groupby(['conversation_id'])['part_id'].first()

conversation_id
10065     0
10106     0
10222     0
10308     0
10342     0
10350     0
10404     0
100120    0
102013    0
102837    0
Name: part_id, dtype: int64

In [324]:
conversations_start['id'].unique()

array([ 10065,  10106,  10222,  10308,  10342, 100120, 102013, 102837,
        10350,  10404], dtype=int64)

In [366]:
consolidated_messages_df.head(5)

,id,user_id,conversation_id,email,message,message_type,part_id,created_at
0,1,4,10065,alexandra.richardson@example.com,Memory support seek ok size. Politics generati...,open,0,1679498647
1,2,4,10065,robert.flores@example.com,Memory support seek ok size. Politics generati...,comment,20482598151,1679498651
2,3,4,10065,anthony.callahan@example.com,Both necessary test available. Until continue ...,assignment,20482622915,1679498716
3,4,4,10065,alexandra.richardson@example.com,Turn important similar arrive. Marriage agree ...,comment,20482707054,1679498908
4,5,4,10065,anthony.callahan@example.com,Measure anything production cup explain well. ...,comment,20482788524,1679499097


In [ ]:
consolidated_messages_df[['user_id','conversation_id']].value_counts()

user_id  conversation_id
1        100120             19
3        102013             13
10       102837             13
7        10106              12
5        10222              10
4        10065               9
0        10404               5
2        10350               5
11       10342               5
12       10308               4
Name: count, dtype: int64

## Data Model 

In [ ]:
CREATE TABLE fact_messages_all
(
    id INTEGER  PRIMARY KEY,
    user_id INTEGER ,
    conversation_id INTEGER ,
    part_id INTEGER,
    FOREIGN KEY (user_id) REFERENCES users(id),
    FOREIGN KEY (part_id) REFERENCES dim_conversation_parts(part_id)  
    
);

CREATE TABLE users (
    id INTEGER  PRIMARY KEY, 
    name TEXT, 
    email TEXT, 
    is_customer INTEGER   -- just 1 or 0
);

CREATE TABLE dim_conversation_parts (
    part_id INTEGER PRIMARY KEY,        -- 
    email TEXT ,
    message TEXT,
    message_type TEXT,
    created_at DATETIME
);



